In [1]:
# simplest data to work with : tiny shakespare 
with open('input.txt', 'r') as f:
    text = f.read()
data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(tokens)

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13, 198, 198, 5962, 22307, 25, 198, 1639, 389, 477, 12939, 2138, 284, 4656, 621, 284, 1145, 680, 30, 198, 198, 3237, 25, 198, 4965, 5634, 13, 12939, 13, 198, 198, 5962, 22307, 25, 198, 5962, 11, 345, 760, 327, 1872, 385, 1526, 28599, 318, 4039, 4472, 284, 262, 661, 13, 198, 198, 3237, 25, 198, 1135, 760, 470, 11, 356, 760, 470, 13, 198, 198, 5962, 22307, 25, 198, 5756, 514, 1494, 683, 11, 290, 356, 1183, 423, 11676, 379, 674, 898, 2756, 13, 198, 3792, 470, 257, 15593, 30, 198, 198, 3237, 25, 198, 2949, 517, 3375, 319, 470, 26, 1309, 340, 307, 1760, 25, 1497, 11, 1497, 0, 198, 198, 12211, 22307, 25, 198, 3198, 1573, 11, 922, 4290, 13, 198, 198, 5962, 22307, 25, 198, 1135, 389, 17830, 3595, 4290, 11, 262, 1458, 1173, 1547, 922, 13, 198, 2061, 4934, 969, 5036, 896, 319, 561, 26958, 514, 25, 611, 484, 198, 19188, 7800, 514, 475, 262, 48713, 414, 11, 981, 340, 547, 198, 

In [5]:
# now we want to take these tokens and forward them in our GPT model 
# to do that we have reshape our long list of token sequence into a (B, T) tensor 
# before that tha shape our forward function takes
# take an example of 24 tokens to understand this

import torch
buf = torch.tensor(tokens[:24])
x = buf.view(4, 6)
x 

tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])

- So, that's our input to the transformer. Now, we also need labels to calucate the loss function. 
- For this we can write some code in the forward pass of the GPT model, as we know the next sequence is just the right of us. But the problem with that approch is that the for the token 13 (the last toke) we don't have the label information. 
- So Andrej fav way to do this, is that we will create the another tensor label of exact same size as x but it contains the labels at every position. 


In [7]:
# so, in the buf we get the last token + 1 and then index our x and y 
buf = torch.tensor(tokens[:24 + 1 ])
x = buf[:-1].view(4, 6) 
y = buf[1:].view(4, 6) 
x, y 

(tensor([[ 5962, 22307,    25,   198,  8421,   356],
         [ 5120,   597,  2252,    11,  3285,   502],
         [ 2740,    13,   198,   198,  3237,    25],
         [  198,  5248,   461,    11,  2740,    13]]),
 tensor([[22307,    25,   198,  8421,   356,  5120],
         [  597,  2252,    11,  3285,   502,  2740],
         [   13,   198,   198,  3237,    25,   198],
         [ 5248,   461,    11,  2740,    13,   198]]))

- Now we go back to our script and create a very simpel dataloader object to load these tokens, feed them to transformer and calculate the loss. 

----- 

Now, we want to calculate the loss function and to do so we have to modify the forward pass

using device: cuda
Only loading first 1k charchters as model input
logits shape torch.Size([4, 32, 50257])
loss tensor(11.0371, device='cuda:0', grad_fn=<NllLossBackward0>)

In [13]:
import math
1/ 50257, -math.log(50257)

(1.9897725689953638e-05, -10.82490511970208)